<a href="https://colab.research.google.com/github/linusms/Hands-on/blob/main/chapter_4_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# 조기종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀 구현(사이킷런 사용하지 말고!)

from sklearn import datasets
iris = datasets.load_iris()

# 데이터 읽기
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

In [15]:
# 선형회귀모형을 위한 샘플에 편향 추가 과정

import numpy as np

X_with_bias = np.c_[np.ones([len(X), 1]), X]

In [16]:
# 훈련 / 테스트 / 검증 세트 만들기(수동)

test_ratio = 0.2                                         # 테스트 세트 비율 = 20%
validation_ratio = 0.2                                   # 검증 세트 비율 = 20%
total_size = len(X_with_bias)                            # 전체 데이터셋 크기

test_size = int(total_size * test_ratio)                 
validation_size = int(total_size * validation_ratio)     
train_size = total_size - test_size - validation_size 

In [17]:
# 인덱스를 섞어서 무작위 섞기 효과 만들기

rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]

X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]

X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [18]:
# 원-핫 인코딩 함수

def to_one_hot(y):
    n_classes = y.max() + 1                 # 클래스 수
    m = len(y)                              # 샘플 수
    Y_one_hot = np.zeros((m, n_classes))    # (샘플 수, 클래스 수) 0-벡터 생성
    Y_one_hot[np.arange(m), y] = 1          # 샘플 별로 해당 클래스의 값만 1로 변경. (넘파이 인덱싱 활용)
    return Y_one_hot

In [19]:
y_train[:5]


array([1, 1, 2, 0, 0])

In [20]:
to_one_hot(y_train[:5])

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [21]:
# 데이터들 원 핫 인코딩으로 변환 - 실제로는 ColumnTransformer와 OneHotEncoder로 변환

Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [22]:
# 소프트맥스 함수 구현


def softmax(logits):
    exps = np.exp(logits)                            # 항목별 지수함수 적용
    exp_sums = np.sum(exps, axis=1, keepdims=True)   # 샘플별 클래스 점수 합산
    return exps / exp_sums                           # 샘플별 소프트맥스 점

In [23]:
# 경사하강법을 위한 초기 파라미터 출발점 설정 

n_inputs = X_train.shape[1] # 샘플의 특성 개수           
n_outputs = len(np.unique(y_train)) # 레이블(답)의 종류, 즉 클래스

Theta = np.random.randn(n_inputs, n_outputs) # 특성*클래스의 차원을 가지는 임의의 수 행렬 - 임의의 초기 파라미터

In [24]:
#  배치 경사하강법 구현

eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

for iteration in range(n_iterations):     # 5001번 반복 훈련
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    
    if iteration % 500 == 0:              # 500 에포크마다 손실(비용) 계산해서 출력
        loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        print(iteration, loss)
    
    error = Y_proba - Y_train_one_hot     # 그레이디언트 계산.
    gradients = 1/m * X_train.T.dot(error)
    
    Theta = Theta - eta * gradients       # 파라미터 업데이트

0 7.449144803178437
500 0.8863971352777716
1000 0.7192393552958617
1500 0.6196005409425926
2000 0.5558037827803247
2500 0.5113947078866622
3000 0.4782778049546998
3500 0.45226696752217677
4000 0.43103512977043257
4500 0.41319545562765264
5000 0.3978699420675774


In [25]:
# 배치 경사하강법으로 구한 로짓으로 예측한 클래스 정확도 확인

logits = X_valid.dot(Theta)              
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)          # 가장 높은 확률을 갖는 클래스 선택

accuracy_score = np.mean(y_predict == y_valid)  # 정확도 계산
accuracy_score

0.8666666666666667

In [30]:
# 규제 추가한 경사하강법

eta = 0.1
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1        # 규제 하이퍼파라미터

Theta = np.random.randn(n_inputs, n_outputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    
    # 반복이 500의 배수일 때마다(적당히) 소프트맥스의 비용함수 값 출력해 정확도 비교
    if iteration % 500 == 0:
        xentropy_loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
        l2_loss = 1/2 * np.sum(np.square(Theta[1:]))  # 편향은 규제에서 제외
        loss = xentropy_loss + alpha * l2_loss        # l2 규제가 추가된 손실
        print(iteration, loss)
    
    # 경사하강도 그래디언트 계산
    error = Y_proba - Y_train_one_hot

    # l2 규제의 그레이디언트(l2 규제항을 theta에 대해 미분한 값).
    l2_loss_gradients = np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]   
    
    # error는 예측값-실제값, 즉 (X*(theta)-y)와 같음, 앞의 1/m은 원래 2/m인데 학습률(eta)에 포함시킨듯
    gradients = 1/m * X_train.T.dot(error) + l2_loss_gradients
    
    Theta = Theta - eta * gradients

0 3.3737297212510353
500 0.5429358459232815
1000 0.5161239488722624
1500 0.5080465739661083
2000 0.5050825279410205
2500 0.5039041571310088
3000 0.5034159640174234
3500 0.5032088705596761
4000 0.5031197515914756
4500 0.5030810541508832
5000 0.5030641537450202


In [27]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333

In [28]:
# 조기종료 추가한 경사하강법

eta = 0.1 
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1            # 규제 하이퍼파라미터
best_loss = np.infty   # 최소 손실값 기억 변수

Theta = np.random.randn(n_inputs, n_outputs)  # 파라미터 새로 초기화

for iteration in range(n_iterations):
    # 훈련 및 손실 계산
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    error = Y_proba - Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

    # 검증 세트에 대한 손실 계산
    logits = X_valid.dot(Theta)
    Y_proba = softmax(logits)
    xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
    l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
    loss = xentropy_loss + alpha * l2_loss
    
    # 500 에포크마다 검증 세트에 대한 손실 출력
    if iteration % 500 == 0:
        print(iteration, loss)
        
    # 에포크마다 최소 손실값 업데이트
    if loss < best_loss:
        best_loss = loss
    else:                                      # 에포크가 줄어들지 않으면 바로(배치 경사 하강법이므로) 훈련 종료
        print(iteration - 1, best_loss)        # 종료되지 이전 에포크의 손실값 출력
        print(iteration, loss, "조기 종료!")
        break

0 3.800479667911591
500 0.5586397578807875
1000 0.5238444116479837
1500 0.5155739029142887
2000 0.5126561556851561
2500 0.5114654681115299
3000 0.5109381922868053
3500 0.5106905627709066
4000 0.5105681713225404
4500 0.5105046505950571
5000 0.5104700903933048


In [29]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9333333333333333